<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-02 11:46:14
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -18.16 K (-0.13%)
Current PnL: -13.70 L (-9.55%)
CY Booked + Current PnL: -6.96 L (-4.85%)
-------------------
Total profit:  4.53 L
Total loss:  -18.23 L
-------------------
Total Booked + Current PnL: 20.02 L (17.18%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.13 L (52.96%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,55.19,M-SC,92.71,134973.0,-7506.0,8125.0,1.56,-5.27,6.02,0.44,243.0,-0.92,0.99,59.31,OX40N,NTT,BANKS
88,UNIONBANK,123.87,163.00,1.69,M-LC,45.83,176713.0,35873.0,8624.0,0.50,25.47,4.88,31.59,89.0,4.16,1.30,53.50,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102772.0,24410.0,9619.0,0.03,31.15,9.36,43.43,77.0,2.54,0.75,38.13,XY24,NTT,MISC
39,HINDALCO,651.95,761.55,-19.08,H-LC,88.54,111880.0,7568.0,9969.0,0.71,7.26,8.91,16.81,27.0,0.76,0.82,24.47,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-17.40,H-LC,82.29,112530.0,8964.0,11748.0,0.08,8.65,10.44,20.00,30.0,0.76,0.83,23.95,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.2,-29.18,L-SC,100.0,142297.0,455.0,56122.0,-1.52,0.32,39.44,39.88,259.0,0.01,1.05,44.21,XY24,NTT,POWER


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,10.42,17435.0,-32158.0,1083.0,8.33,-64.84,6.21,-62.66,NaN,-29.69,0.13,16.92,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,3.12,44952.0,3697.0,7107.0,5.83,8.96,15.81,26.19,1.0,0.52,0.33,5.83,XY24,ATH,IT
63,QUESS,296.83,986.00,-31.13,M-SC,30.21,68394.0,3388.0,147540.0,4.05,5.21,215.72,232.18,191.0,0.02,0.50,4.73,XY24,NTT,MISC
78,SURYODAY,189.78,240.00,60.09,H-SC,16.67,113199.0,-36727.0,76398.0,2.17,-24.50,67.49,26.46,145.0,-0.48,0.83,45.02,XR,NTT,BANKS
60,NATIONALUM,189.63,247.44,-48.64,H-MC,77.08,102905.0,2401.0,28237.0,2.04,2.39,27.44,30.49,97.0,0.09,0.76,35.57,MH,ATH,METALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,INDUSINDBK,1354.50,1800.0,80.59,L-MC,72.92,48980.0,-28226.0,53618.0,-2.25,-36.56,109.47,32.89,253.0,-0.53,0.36,34.89,XR,NTT,BANKS
69,SAMMAANCAP,170.35,326.0,-171.59,M-SC,89.58,82428.0,-19782.0,113174.0,-2.16,-19.35,137.30,91.37,206.0,-0.17,0.61,33.81,XY25,NTT,FINANCE
32,ENRIN,1377.95,2975.9,NaN,NaN,73.96,72752.0,38303.0,1644.0,-2.07,111.19,2.26,115.97,NaN,23.30,0.53,11.25,AR,ATH,ELECTRICAL
24,COFFEEDAY,59.14,80.0,-53.72,L-SC,97.92,68851.0,-44698.0,84749.0,-1.97,-39.36,123.09,35.27,261.0,-0.53,0.51,68.52,XR,NTT,HOTELS
30,DMART,3764.92,5201.0,-14.84,X-LC,81.25,213860.0,29379.0,40997.0,-1.97,15.93,19.17,38.14,15.0,0.72,1.57,28.22,X40N,NTT,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,55.19,M-SC,92.71,134973.0,-7506.0,8125.0,1.56,-5.27,6.02,0.44,243.0,-0.92,0.99,59.31,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-213.31,H-SC,96.88,81695.0,-13621.0,13659.0,-1.56,-14.29,16.72,0.04,154.0,-1.00,0.60,61.13,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-32.03,M-SC,5.21,144860.0,-26281.0,26220.0,-1.14,-15.36,18.10,-0.03,189.0,-1.00,1.06,4.11,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,123.77,H-SC,87.50,146680.0,-27879.0,27913.0,0.20,-15.97,19.03,0.02,127.0,-1.00,1.08,27.19,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.66,X-MC,14.58,85400.0,-17252.0,17251.0,-1.75,-16.81,20.20,-0.00,58.0,-1.00,0.63,29.82,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,3.12,44952.0,3697.0,7107.0,5.83,8.96,15.81,26.19,1.0,0.52,0.33,5.83,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.74,X-LC,41.67,268086.0,-27848.0,121765.0,0.21,-9.41,45.42,31.73,1.0,-0.23,1.97,7.30,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,59.38,288351.0,26750.0,118887.0,0.26,10.23,41.23,55.67,2.0,0.23,2.12,16.95,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.84,X-LC,21.88,257477.0,-12870.0,47505.0,0.15,-4.76,18.45,12.81,6.0,-0.27,1.89,7.68,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.84,X-LC,28.12,279466.0,14040.0,42870.0,-0.89,5.29,15.34,21.44,9.0,0.33,2.05,11.81,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,1.69,M-LC,45.83,176713.0,35873.0,8624.0,0.50,25.47,4.88,31.59,89.0,4.16,1.30,53.50,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,26.04,102772.0,24410.0,9619.0,0.03,31.15,9.36,43.43,77.0,2.54,0.75,38.13,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,73.96,72752.0,38303.0,1644.0,-2.07,111.19,2.26,115.97,NaN,23.30,0.53,11.25,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,94.40,M-MC,58.33,233383.0,8421.0,159237.0,1.64,3.74,68.23,74.53,185.0,0.05,1.71,35.50,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,27.50,H-SC,69.27,130662.0,5365.0,53846.0,-1.96,4.28,41.21,47.25,139.0,0.10,0.96,63.47,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.09,H-SC,53.12,119126.0,17122.0,27899.0,0.27,16.79,23.42,44.14,124.0,0.61,0.88,35.75,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102772.0,24410.0,9619.0,0.03,31.15,9.36,43.43,77.0,2.54,0.75,38.13,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,1.69,M-LC,45.83,176713.0,35873.0,8624.0,0.50,25.47,4.88,31.59,89.0,4.16,1.30,53.50,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,73.96,72752.0,38303.0,1644.0,-2.07,111.19,2.26,115.97,NaN,23.30,0.53,11.25,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,93.75,205344.0,33723.0,128340.0,1.27,19.65,62.50,94.43,198.0,0.26,1.51,51.56,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,75.00,164703.0,13758.0,95692.0,0.42,9.11,58.10,72.51,69.0,0.14,1.21,13.45,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.52,H-SC,91.67,136671.0,13335.0,121501.0,-0.74,10.81,88.90,109.32,122.0,0.11,1.00,35.02,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,27.50,H-SC,69.27,130662.0,5365.0,53846.0,-1.96,4.28,41.21,47.25,139.0,0.10,0.96,63.47,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,10.42,17435.0,-32158.0,1083.0,8.33,-64.84,6.21,-62.66,NaN,-29.69,0.13,16.92,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,97.92,68851.0,-44698.0,84749.0,-1.97,-39.36,123.09,35.27,261.0,-0.53,0.51,68.52,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.25,H-SC,67.71,92985.0,-4995.0,27728.0,0.11,-5.10,29.82,23.21,137.0,-0.18,0.68,34.86,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,5.75,X-SC,98.96,98444.0,-2519.0,97046.0,-0.58,-2.50,98.58,93.63,64.0,-0.03,0.72,46.63,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102772.0,24410.0,9619.0,0.03,31.15,9.36,43.43,77.0,2.54,0.75,38.13,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142297.0,455.0,56122.0,-1.52,0.32,39.44,39.88,259.0,0.01,1.05,44.21,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,5.75,X-SC,98.96,98444.0,-2519.0,97046.0,-0.58,-2.50,98.58,93.63,64.0,-0.03,0.72,46.63,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,97.92,68851.0,-44698.0,84749.0,-1.97,-39.36,123.09,35.27,261.0,-0.53,0.51,68.52,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-213.31,H-SC,96.88,81695.0,-13621.0,13659.0,-1.56,-14.29,16.72,0.04,154.0,-1.00,0.60,61.13,OX40N,NTT,MISC
51,JIOFIN,310.58,387.00,6.94,H-LC,95.83,229215.0,11809.0,41694.0,-0.65,5.43,18.19,24.61,95.0,0.28,1.68,62.98,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.84
1,25,41.90
2,50,70.45


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.51
LC    34.03
MC    21.48
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.57
X40      12.23
XY25     12.01
X40N     11.81
XR       10.18
AR        8.22
OX40N     8.17
MH        1.72
X5K       1.48
X200      1.41
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.69
M-SC    16.00
H-LC    14.12
X-LC    12.18
H-MC     9.50
X-MC     7.50
M-LC     6.68
X-SC     4.34
M-MC     4.12
L-SC     2.48
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.07,-2.02,36.15
IT,12.10,-10.41,64.63
BANKS,8.05,-4.04,46.49
FINANCE,7.91,-12.05,52.60
MISC,6.65,-14.84,68.13
PAINTS,5.65,-13.31,35.65
ELECTRICAL,5.45,0.72,45.65
HEALTHCARE,4.51,-5.03,35.71
AUTO,4.31,-15.00,65.57


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2703959.0,27
XR,1073569.0,13
AR,1024633.0,10
X40,655571.0,9
X40N,557690.0,11
XY25,484132.0,8
OX40N,318383.0,11
SR,178871.0,2
MH,82083.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969928.0,20
M-SC,1764158.0,21
H-LC,616322.0,12
H-MC,539347.0,8
X-LC,517121.0,7
X-MC,433089.0,6
M-MC,409080.0,3
X-SC,328431.0,5
M-LC,263947.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      861263.0      6
M-SC       XY24      814970.0      7
H-SC       AR        522450.0      3
M-SC       XR        333047.0      4
M-MC       XY24      302891.0      2
H-MC       XY24      300064.0      3
X-LC       X40       240652.0      2
H-SC       X40N      225600.0      4
           XR        214371.0      3
H-LC       X40N      200823.0      4
M-SC       AR        182355.0      2
           OX40N     166621.0      5
H-LC       X40       159180.0      3
M-LC       XY24      155984.0      3
X-MC       X40       154064.0      2
L-SC       XR        146746.0      2
X-SC       XR        123906.0      1
X-LC       XY25      119036.0      3
           AR        116436.0      1
M-SC       XY25      113174.0      1
M-MC       XR        106189.0      1
X-MC       XY25      105792.0      1
           XY24       97415.0      1
X-SC       SR         97046.0      1
M-LC       XR         95692.0      1
H-SC       OX40N      92398.0      3
H-MC       XY25       87529.0      1
M-SC       SR         81825.0      1
H-LC       AR         80053.0      1
M-SC       X40        72166.0      1
H-LC       X200       68775.0      1
           X5K        65797.0      2
X-SC       XY24       65366.0      1
H-MC       AR         62305.0      1
L-SC       AR         59390.0      1
X-MC       X40N       58567.0      1
L-SC       XY24       56122.0      1
H-SC       MH         53846.0      1
L-MC       XR         53618.0      1
L-LC       XY25       46330.0      1
X-SC       OX40N      42113.0      2
H-LC       XY24       41694.0      1
X-LC       X40N       40997.0      1
H-MC       X40N       31703.0      1
           X40        29509.0      1
           MH         28237.0      1
X-MC       OX40N      17251.0      1
M-LC       XY25       12271.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
